In [28]:
!wget -O wikitext-filtered-full.zip "https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1"
!wget -O wikitext-filtered-10k.zip "https://www.dropbox.com/scl/fi/ek174r3sg7qjx0aa9atop/wikitext-filtered-10k.zip?rlkey=zy6jqxv6qsc16lr9qm3ki9uhf&dl=1"

--2025-10-17 01:16:38--  https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uca44f920c83c031ae0f3bae645d.dl-eu.dropboxusercontent.com/cd/0/inline/CzX67oJGTojBPIvunOrL_WLBF2TQjIgWaFvApy9z4FkJub4V6rPKc8sIleX6_c8C5TALZ3-s41E_oADSdM1k2ilYe2Sl7yFfFsmqYynbG3OrpidcFvDiNIL1TKCnhQwSv4Az608PDVfA3NlFeNOUtDuz/file?dl=1# [following]
--2025-10-17 01:16:39--  https://uca44f920c83c031ae0f3bae645d.dl-eu.dropboxusercontent.com/cd/0/inline/CzX67oJGTojBPIvunOrL_WLBF2TQjIgWaFvApy9z4FkJub4V6rPKc8sIleX6_c8C5TALZ3-s41E_oADSdM1k2ilYe2Sl7yFfFsmqYynbG3OrpidcFvDiNIL1TKCnhQwSv4Az608PDVfA3NlFeNOUtDuz/file?dl=1
Resolving uca44f920c83c031ae0f3bae645d.dl-eu.dropboxusercontent.com (uca44f920c83c031ae0f3bae645d.dl-eu.dro

In [29]:
!unzip wikitext-filtered-full.zip
!unzip wikitext-filtered-10k.zip

Archive:  wikitext-filtered-full.zip
replace wikitext-filtered-full/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikitext-filtered-full/state.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikitext-filtered-full/data-00000-of-00001.arrow? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  wikitext-filtered-10k.zip
replace wikitext-filtered-10k/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikitext-filtered-10k/state.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikitext-filtered-10k/data-00000-of-00001.arrow? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [30]:
!pip install datasets

In [31]:
!pip install gensim

In [54]:
import os, string, nltk, numpy as np, pandas as pd
from datasets import Dataset
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr
from nltk.corpus import stopwords # Import stopwords from nltk.corpus


#Download stopwords
try:
    stopwords_list = stopwords.words('english') # Get the list of English stopwords
except LookupError:
    nltk.download('stopwords')
    stopwords_list = stopwords.words('english') # Get the list of English stopwords after downloading

STOP = set(stopwords_list) # Convert the list to a set for efficient lookup

In [55]:
def load_wikitext():
    small_dir = "wikitext-filtered-10k"
    large_dir = "wikitext-filtered-full"
    ds_small = Dataset.load_from_disk(small_dir)
    ds_large = Dataset.load_from_disk(large_dir)
    print(f"wikitext_small: {len(ds_small)} docs | wikitext_large: {len(ds_large)} docs")
    return ds_small, ds_large

wikitext_small, wikitext_large = load_wikitext()

wikitext_small: 10000 docs | wikitext_large: 859955 docs


In [56]:
def preprocess_text(text: str):
    # spec-compliant: lowercase, space-tokenize, remove punctuation, remove stopwords
    tokens = text.lower().split()
    tokens = [t.translate(str.maketrans('', '', string.punctuation)) for t in tokens]
    tokens = [t for t in tokens if t and t not in STOP]
    return tokens

def prepare_corpus(dataset):
    corpus = []
    for ex in dataset:
        txt = ex.get("text", "").strip()
        if not txt:
            continue
        toks = preprocess_text(txt)
        if toks:
            corpus.append(toks)
    return corpus


print("Preparing corpus (smal")
train_small = prepare_corpus(wikitext_small)
print("Docs (small):", len(train_small))
print("Sample tokens:", train_small[0][:20])



Preparing corpus (smal
Docs (small): 9998
Sample tokens: ['senjō', 'valkyria', '3', 'unrecorded', 'chronicles', 'japanese', '戦場のヴァルキュリア3', 'lit', 'valkyria', 'battlefield', '3', 'commonly', 'referred', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', 'tactical', 'role']


In [62]:
SEED = 42
WORKERS = os.cpu_count()

print("\nTraining Word2Vec (small)")
model_small = Word2Vec(
    sentences=train_small,
    vector_size=50,
    window=5,
    min_count=5,
    sg=1,                 # <- Skip-gram
    workers=WORKERS,
    seed=SEED,
    epochs=5
)
print("Vocab size:", len(model_small.wv))


Training Word2Vec (small)
Vocab size: 14555


In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
# Import WordSim csv

import pandas as pd

file_path = '/content/drive/My Drive/wordsim353/combined.csv'

try:
    wordsim_df = pd.read_csv(file_path)
    print("WordSim-353 data loaded successfully.")
    display(wordsim_df.head())
except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")
    print("Please check the file path in your Google Drive and update the 'file_path' variable.")
except Exception as e:
    print(f"An error occurred while loading the file: {e}")

WordSim-353 data loaded successfully.


,Word 1,Word 2,Human (mean)
0,love,sex,6.77
1,tiger,cat,7.35
2,tiger,tiger,10.00
3,book,paper,7.46
4,computer,keyboard,7.62


In [64]:
def cos_sim(model, w1, w2):
    w1, w2 = w1.lower(), w2.lower()
    try:
        return model.wv.similarity(w1, w2)
    except KeyError:
        return np.nan

# Step 3(a): four pairs
pairs = [("plane","car"),("planet","sun"),("cup","article"),("sugar","approach")]
print("\n-- Cosines (wikitext_small, sg=1, seed=42) --")
for a, b in pairs:
    s = cos_sim(model_small, a, b)
    print(f"{a:7s}/{b:8s} → {s:.4f}" if s==s else f"{a}/{b} → OOV")

# Full list for Step 4
wordsim_df['cosine_small_sg1'] = [cos_sim(model_small, a, b) for a, b in zip(wordsim_df['Word 1'], wordsim_df['Word 2'])]

# Spearman (drop OOVs)
df_clean = wordsim_df.dropna(subset=['cosine_small_sg1'])
if 'Human (mean)' in df_clean.columns:
    rho, p = spearmanr(df_clean['Human (mean)'], df_clean['cosine_small_sg1'])
    print(f"\nSpearman ρ (small, sg=1): {rho:.4f}  (p={p:.4g})")
else:
    print("Column 'Human (mean)' not found in CSV. Check header names.")


-- Cosines (wikitext_small, sg=1, seed=42) --
plane  /car      → 0.6338
planet /sun      → 0.8940
cup    /article  → 0.3744
sugar  /approach → 0.8594

Spearman ρ (small, sg=1): 0.1970  (p=0.001384)


In [59]:
from scipy.stats import spearmanr

# Drop rows with NaN in the 'Word2Vec_Similarity_Small' column as spearmanr cannot handle NaNs
wordsim_df_cleaned = wordsim_df.dropna(subset=['Word2Vec_Similarity_Small'])

# Get the human scores and the computed similarities from the cleaned DataFrame
human_scores = wordsim_df_cleaned['Human (mean)']
computed_similarities = wordsim_df_cleaned['Word2Vec_Similarity_Small']

# Compute the Spearman rank-order correlation coefficient
spearman_corr, p_value = spearmanr(human_scores, computed_similarities)

print(f"Spearman rank-order correlation coefficient: {spearman_corr:.4f}")
print(f"P-value: {p_value:.4f}")

Spearman rank-order correlation coefficient: 0.0566
P-value: 0.3628
